In [1]:
pip install linearmodels

  Obtaining dependency information for linearmodels from https://files.pythonhosted.org/packages/5e/54/39a20f68c49ec8329db26ef7628a54728f0c4cfb4d7bbae8e87d9f662dce/linearmodels-5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyhdfe>=0.1 from https://files.pythonhosted.org/packages/2f/51/cb006fbc08c32f161035fb19ca718250eb5f6d0692ea6dcc1e62c3e556a2/pyhdfe-0.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for formulaic>=0.6.5 from https://files.pythonhosted.org/packages/ae/f0/ff275c0009f0037c381c6e3a94049b9792df7d7f437c5ccd0c7aefca5f22/formulaic-0.6.6-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import RandomEffects
from sklearn.preprocessing import MinMaxScaler

In [116]:
panel_data = pd.read_csv('Rental.csv')

panel_data.head()

,index,borough,price_long_rents,count_long_rents,year,review_scores_airbnb,price_airbnb,borough_area,count_airbnb,density_airbnb,code,increasing_ratio_countrents,increasing_ratio_priceairbnb,increasing_ratio_countairbnb
0,2023,Barking and Dagenham,1350.025316,790,2023,4.551263,245.840426,36107810,564,1.561989,1,-0.210000,1.549919,0.302540
1,2023,Barnet,1701.178082,2190,2023,4.602639,160.523344,86748314,2249,2.592558,2,-0.252560,0.032513,0.396027
2,2023,Bexley,1399.245283,530,2023,4.609408,95.046465,60580233,495,0.817098,3,-0.485437,0.059768,0.204380
3,2023,Brent,1628.101695,590,2023,4.580713,179.079197,43232640,2740,6.337804,4,-0.508333,-0.132528,0.324952
4,2023,Bromley,1499.208696,2300,2023,4.685929,105.972798,150134858,772,0.514204,5,-0.083665,-0.060045,0.299663


In [111]:
panel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         132 non-null    int64  
 1   borough                       132 non-null    object 
 2   price_long_rents              132 non-null    float64
 3   count_long_rents              132 non-null    int64  
 4   year                          132 non-null    int64  
 5   review_scores_airbnb          132 non-null    float64
 6   price_airbnb                  132 non-null    float64
 7   borough_area                  132 non-null    int64  
 8   count_airbnb                  132 non-null    int64  
 9   density_airbnb                132 non-null    float64
 10  code                          132 non-null    int64  
 11  increasing_ratio_countrents   132 non-null    float64
 12  increasing_ratio_priceairbnb  132 non-null    float64
 13  incre

In [117]:
panel_data = panel_data.set_index(["index", "code"])

In [118]:
panel_data.columns.to_list()

['borough',
 'price_long_rents',
 'count_long_rents',
 'year',
 'review_scores_airbnb',
 'price_airbnb',
 'borough_area',
 'count_airbnb',
 'density_airbnb',
 'increasing_ratio_countrents',
 'increasing_ratio_priceairbnb',
 'increasing_ratio_countairbnb']

In [120]:
# 选择自变量和因变量

independent_vars = [
    "density_airbnb",
    "price_airbnb",
    "increasing_ratio_priceairbnb",
    "count_airbnb"
]
dependent_var = "price_long_rents"  # 之前计算的综合指数

# 准备模型的因变量和自变量
y = panel_data[dependent_var]
X = panel_data[independent_vars]

# 添加常数项
X = sm.add_constant(X)

# 构建随机效应模型
model = RandomEffects(y, X)
results = model.fit()

# 输出模型结果
print(results)

                        RandomEffects Estimation Summary                        
Dep. Variable:       price_long_rents   R-squared:                        0.8823
Estimator:              RandomEffects   R-squared (Between):              0.9906
No. Observations:                 132   R-squared (Within):               0.8774
Date:                Thu, Dec 07 2023   R-squared (Overall):              0.8823
Time:                        00:56:51   Log-likelihood                   -840.98
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      237.93
Entities:                           4   P-value                           0.0000
Avg Obs:                       33.000   Distribution:                   F(4,127)
Min Obs:                       33.000                                           
Max Obs:                       33.000   F-statistic (robust):             237.93
                            